# Use Case 3: Associating Clinical Variables with Acetylation

For this use case we will show how to analyze the acetylation data with a clinical attribute. We will use the clinical attribute "histologic_type", but you can apply the processes shown here to many other clinical attributes. Our goal is to identify which acetylation sites differ significantly in frequency between non-tumor, serous and endometrial cells.

# Step 1: Import Packages and Load Data

We will start by importing the data analysis tools we need, importing the cptac package, and loading the Endometrial dataset.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import statsmodels.stats.multitest
import matplotlib.pyplot as plt
import seaborn as sns
import math
import cptac
import cptac.utils as ut

en = cptac.Ucec()

# Step 2: Choose Clinical Attribute and Join Dataframes

For this use case, we will use the 'histologic_type' clinical attribute in order to find differences in acetylation sites between "endometrioid" and "serous" cancer cells. 

In [2]:
#Set desired attribute to variable 'clinical_attribute'
clinical_attribute = "histologic_type"
en.list_data_sources()

,Data type,Available sources
0,CNV,washu
1,acetylproteomics,umich
2,circular_RNA,bcm
3,clinical,mssm
4,deconvolution_cibersort,washu
5,deconvolution_xcell,washu
6,follow-up,mssm
7,medical_history,mssm
8,miRNA,washu
9,phosphoproteomics,umich


Here we will join our desired clinical attribute with our acetylation dataframe using the `en.join_metadata_to_omics` method.

In [3]:
#Join attribute with acetylation dataframe
clinical_and_acetylation = en.multi_join({"mssm clinical": clinical_attribute, "umich acetylproteomics": ''}, levels_to_drop=['Peptide', 'Database_ID'])

# Use the cptac.utils.reduce_multiindex function to combine the
# multiple column levels, so it's easier to graph our data
clinical_and_acetylation = ut.reduce_multiindex(df=clinical_and_acetylation, flatten=True)

clinical_and_acetylation.head()

cptac warning: Due to dropping the specified levels, dataframe now has 584 duplicated column headers. (C:\Users\sabme\anaconda3\lib\site-packages\cptac\cancers\cancer.py, line 587)


Name,histologic_type,ARF5_umich_acetylproteomics_K109,FKBP4_umich_acetylproteomics_K181,FKBP4_umich_acetylproteomics_K213,FKBP4_umich_acetylproteomics_K234,FKBP4_umich_acetylproteomics_K266,FKBP4_umich_acetylproteomics_K294,FKBP4_umich_acetylproteomics_K35,FKBP4_umich_acetylproteomics_K354,FKBP4_umich_acetylproteomics_K76,...,PDIA4_umich_acetylproteomics_K570,PDIA4_umich_acetylproteomics_K611,PDIA4_umich_acetylproteomics_K637,AC004706.3_umich_acetylproteomics_K21,K102,K105,WIZ_umich_acetylproteomics_K16,WIZ_umich_acetylproteomics_K638,K228,K304
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00006,Endometrioid carcinoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.945484,NaN,NaN,NaN,NaN,NaN,22.290219
C3L-00008,Endometrioid carcinoma,NaN,13.420601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,16.790586,NaN,NaN,14.147555,12.664316,NaN,22.236406
C3L-00032,Endometrioid carcinoma,NaN,NaN,15.060923,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.952557,NaN,NaN,NaN,NaN,14.351192,21.437865
C3L-00084,Endometrioid carcinoma,NaN,NaN,NaN,NaN,NaN,12.336663,NaN,NaN,NaN,...,NaN,NaN,12.251376,14.506364,NaN,NaN,NaN,NaN,NaN,25.212475
C3L-00090,Endometrioid carcinoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.483674,NaN,...,NaN,NaN,NaN,16.116837,13.641868,NaN,NaN,11.869961,NaN,21.68762


# Step 3: Format Dataframe to Compare Acetylproteomic Sites Between Histologic Types

In [4]:
clinical_attribute = "histologic_type"
#Show possible variations of histologic_type
clinical_and_acetylation[clinical_attribute].unique()

array(['Endometrioid carcinoma', 'Serous carcinoma',
       'Clear cell carcinoma', 'Mixed cell adenocarcinoma', nan],
      dtype=object)

In this step, we will make two different dataframes for "Endometrioid" and "Serous" cancer types, as well as fill the NaN columns with "Non-Tumor."

In [5]:
#Make dataframes with only endometrioid and only serous data in order to compare 
endom = clinical_and_acetylation.loc[clinical_and_acetylation[clinical_attribute] == "Endometrioid carcinoma"]
serous = clinical_and_acetylation.loc[clinical_and_acetylation[clinical_attribute] == "Serous carcinoma"]
#Here is where we set the NaN values to "Non_Tumor"
clinical_and_acetylation[[clinical_attribute]] = clinical_and_acetylation[[clinical_attribute]].fillna(
    value="Non_Tumor")

Now that we have our different dataframes, we want to make sure that the amount of data we are using for each site is significant. Since there are fewer patients with "serous" tumors than with "endometrioid," we will check to make sure that we have at least five values for each acetylation site that we are comparing that have a measurement of intensity for serous patients. We will remove every acetylation site from our dataframe that doesn't have at least five values among the serous patients.

In [6]:
#Remove every column that doesn't have at least 5 values among the serous patients
print("Total Sites: ", len(serous.columns) - 1)
sites_to_remove = []
for num in range(1, len(serous.columns)):
    serous_site = serous.columns[num]
    one_site = serous[serous_site]
    num_datapoints_ser = one_site.count()
    if num_datapoints_ser < 5:
        sites_to_remove.append(serous_site)

clinical_and_acetylation = clinical_and_acetylation.drop(sites_to_remove, axis = 1)

#Also remove non-tumor patients from our dataframe to use in comparison, as we want to compare only endometrioid and serous types
clinical_and_acetylation_comparison = clinical_and_acetylation.loc[clinical_and_acetylation['histologic_type'] != 'Non_Tumor']
clinical_and_acetylation_comparison = clinical_and_acetylation_comparison.loc[clinical_and_acetylation_comparison['histologic_type'] != 'Mixed cell adenocarcinoma']
clinical_and_acetylation_comparison = clinical_and_acetylation_comparison.loc[clinical_and_acetylation_comparison['histologic_type'] != 'Clear cell carcinoma']


print("Removed: ", len(sites_to_remove))
print("Remaining Sites: ", len(clinical_and_acetylation_comparison.columns) - 1)
print("Adjusted p-value cutoff will be: ", .05/(len(clinical_and_acetylation_comparison.columns)-1))

Total Sites:  11739


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

# Step 4: Compare Endometrioid and Serous Values

We will now call the wrap_ttest method, which will loop through the data and compare endometrioid versus serous data for each acetylation site. If we find a site that is significantly different, we will add it to a dataframe, with its p-value. The default alpha used is .05, which will be adjusted to account for multiple testing using a bonferroni correction, dividing alpha by the number of comparisons that will occur (the number of comparison columns).

In [ ]:
#Make list of all remaining sites in dataframe to pass to wrap_ttest function
columns_to_compare = list(clinical_and_acetylation_comparison.columns)

#Remove the "Histologic_type" column (at index 0) from this list
columns_to_compare = columns_to_compare[1:]

#Perform ttest on each column in dataframe
significant_sites_df = ut.wrap_ttest(df=clinical_and_acetylation_comparison, label_column="histologic_type", comparison_columns=columns_to_compare)

#List significant results
significant_sites_df

# Step 5: Graph Results

Now that we have eight acetylation sites that differ significantly between endometrioid and serous intensities, we will graph a couple of them using a boxplot and a stripplot in order to visually see the difference, as well as compare with normal cells.

In [ ]:
graphingSite = 'FOXA2_umich_acetylproteomics_K274'
print(scipy.stats.ttest_ind(endom[graphingSite], serous[graphingSite]))
sns.boxplot(x=clinical_attribute, y=graphingSite, data=clinical_and_acetylation, showfliers=False, 
            order=["Non_Tumor", "Endometrioid carcinoma", "Serous carcinoma"])
sns.stripplot(x=clinical_attribute, y=graphingSite, data=clinical_and_acetylation, color='.3', 
              order=["Non_Tumor", "Endometrioid carcinoma", "Serous carcinoma"])
plt.show()

In [ ]:
graphingSite = 'TBL1XR1_umich_acetylproteomics_K102'
print(scipy.stats.ttest_ind(endom[graphingSite], serous[graphingSite]))
sns.boxplot(x=clinical_attribute, y=graphingSite, data=clinical_and_acetylation, showfliers = False, 
            order=["Non_Tumor", "Endometrioid carcinoma", "Serous carcinoma"])
sns.stripplot(x=clinical_attribute, y=graphingSite, data=clinical_and_acetylation, color='.3', 
              order=["Non_Tumor", "Endometrioid carcinoma", "Serous carcinoma"])
plt.show()